# Evaluacion de la traducción

## Perplexity
(Fuente: https://www.quora.com/How-does-perplexity-function-in-natural-language-processing)
- Es una medida que indica con cuánta seguridad un modelo de lenguaje predecirá los datos de test (anotados).
Por ejemplo, si a la salida del modelo de lenguaje esperamos la frase:

"I love NLP"

podemos estimar la seguridad del modelo como:

$$ \prod_{i=1}^N p(w_i) = p("NLP" | "I", "love") * p("love" | "I") * p("I") $$

Cuanto mas alto, mas seguro estará el modelo de su predicción.
A medida que la cantidad de palabras en la oración crece, mas probabilidades se multiplican entre sí y este número tiende a mezclarse con la resolución de la máquina. Por ello es conveniente hacer la evaluación en un espacio logarítmico:

$$ \log_b \prod_{i=1}^N p(w_i) = \sum_{i=1}^N \log_b p(w_i) $$

Si normalizamos por la cantidad de palabras, nos independizamos del largo de la oración.  Si además multiplicamos por $-1$ nos queda:

$$ ACE =  - \frac{1}{N} \sum_{i=1}^N \log_b p(w_i) $$

ACE: Average Cross Entropy

Si el modelo de lenguaje está totalmente seguro de las palabras que está dando a su salida, entonces todas las probabilidades serán 1.  
En el caso totalmente aleatorio, el modelo devuelve palabras con distribución uniforme, entonces todas las probabilidades serán $\frac{1}{|V|}$, donde $|V|$ es el tamaño del vocabulario.
En el peor de los casos, el modelo asigna una probabilidad cero a cada una de las palabras que esperamos a la salida.

Se define la perplejidad (perplexity) como:

$$PPL = b^{ACE} = b^{- \frac{1}{N} \sum_{i=1}^N \log_b p(w_i)}$$

Nótese que cuando el modelo está totalmente seguro, la PPL vale cero.  
En el caso totalmente aleatorio, $p=\frac{1}{|V|}$ y por lo tanto la PPL vale $|V|$.  
En el peor de los casos, $p=0$ para todas las palabras y la PPL vale $\infty$.

En el caso de $b=e$ nos queda la categorical crossentropy calculada en Keras.

In [1]:
# Implementación en Keras

def perplexity(y_true, y_pred):
    cross_entropy = K.sparse_categorical_crossentropy(y_true, y_pred)
    perplexity = K.exp(cross_entropy)
    return perplexity

## Bilingual Evaluation Understudy (BLEU Score)
(Fuente: https://machinelearningmastery.com/calculate-bleu-score-for-text-python/)

Sirve para evaluar una oración candidata contra un conjunto de oraciones de referencia.  

### Precisión

Candidata: This this this
Referencia : This is a test

$$ P=\frac{\#NgramaComunes}{\#NgramasCandidata}$$

Para $N=1$:  

$$ P = \frac{3}{3} = 1 $$

Malas traducciones tienen una alta precisión. -> No sirve

### Precisión modificada 

Se tiene en cuenta la cantidad de Ngramas en la oración de referencia, siendo este el límite a la hora de contabilizar las apariciones de la oración candidata.  
Para el caso anterior:  
$$PM = \frac{2}{3} = 0.67 $$  

Candidata: The the  
Referencia: The cat is on the mat.

$$ MP = \frac{2}{2} $$


Sigue teniendo problemas para medir la calidad de una traducción.
Si las frases a comparar tienen una longitud muy distinta, no podemos afirmar que sean similares.  
Si la frase candidata tiene mayor longitud que la de referencia, este aspecto se ve reflejado en la fórmula de precisión modificada anterior. Habrá muchos ngramas en la frase candidata que no aparecerán en la frase de referencia por lo que la precisión será menor.  
Si la frase candidata es mas corta que la de referencia, debemos penalizar la precisión modificada por algún factor que sea menor que uno.


### Bigrams, Trigrams, etc

Candidata: The the cat -> "The the", "the cat"  
Referencia: The cat is on the mat -> "The cat", "cat is", "is on", "on the", "the mat"  

$$ P_2 = \frac{1}{2} $$
$$ MP_2 = \frac{1}{2} $$

### Penalización por brevedad (Brevity Penalty)

$$ BP = \begin{cases} 1 \ \mbox{si} \ c > r \\ e ^{1-\frac{r}{c}}\end{cases} $$

$$ MPBP= MP * BP $$ 

### BLEU Score

Se calcula sobre un rango de Ngrams (Ej: Típicamente de 1 a 4) 

$$BLEU = PB \exp\left(\sum_{n=1}^N w_n \log \text{MP}_n\right)$$ 

$w_n$ es un peso que se le da a cada una de las MP de cada Ngram. Típicamente vale $\frac{1}{N}$.  

In [50]:
# En NLTK:

from nltk.translate.bleu_score import sentence_bleu
reference = [["this","is","test"],["this","is","a","a"]]
candidate = ["this","is","a","test","a"]
score = sentence_bleu(reference, candidate,weights=(0.5,0.5))
print(score)

0.7071067811865476


### BLEU Score en corpus
Para el caso de BLEU Score de un corpus completo se utiliza el micro average precision. El mismo consiste en sumar los casos satisfactorios de todas las oraciones y dividir por la suma de los tamaños de cada oración candidato.

In [52]:
# two references for one document
from nltk.translate.bleu_score import corpus_bleu
references = [[['this', 'is', 'a', 'test'], ['this', 'is' 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = corpus_bleu(references, candidates,weights=(1,))
print(score)

1.0


El BLEU Score en corpus es una medida que tiene muy alta correlación con los puntajes otorgados por humanos a las traducciones. Es por ello que es uno de los mas utilizados. No pasa lo mismo con el BLEU Score en oraciones.